# ResNet

## Installation and import of the libraries

In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np

2024-11-17 03:36:39.493454: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 03:36:39.753149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 03:36:39.849150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 03:36:39.875646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 03:36:40.057787: I tensorflow/core/platform/cpu_feature_guar

## Preprocess

In [2]:
# Cargar el archivo CSV con las características
df = pd.read_csv('wide_data.csv')

# Definir la carpeta donde se encuentran las imágenes
image_folder = '../archive/images/images'

df_subset = df.sample(n=10000, random_state=42)

def preprocess_image(img_path):
    try:
        img = load_img(img_path, target_size=(224, 224))  # Redimensionar la imagen
        img_array = img_to_array(img)  # Convertir la imagen a array
        img_array = preprocess_input(img_array)  # Preprocesar la imagen para ResNet50
        return img_array
    except Exception as e:
        print(f"Error al cargar la imagen {img_path}: {e}")
        return None  # Retornar None si la imagen no se carga correctamente

# Función para cargar las etiquetas con valores nulos reemplazados por 0
def load_labels(row):
    label = [
        row['silhouette_type'] if pd.notna(row['silhouette_type']) else 0,
        row['waist_type'] if pd.notna(row['waist_type']) else 0,
        row['neck_lapel_type'] if pd.notna(row['neck_lapel_type']) else 0,
        row['sleeve_length_type'] if pd.notna(row['sleeve_length_type']) else 0,
        row['toecap_type'] if pd.notna(row['toecap_type']) else 0,
        row['closure_placement'] if pd.notna(row['closure_placement']) else 0,
        row['cane_height_type'] if pd.notna(row['cane_height_type']) else 0,
        row['heel_shape_type'] if pd.notna(row['heel_shape_type']) else 0,
        row['knit_structure'] if pd.notna(row['knit_structure']) else 0,
        row['length_type'] if pd.notna(row['length_type']) else 0,
        row['woven_structure'] if pd.notna(row['woven_structure']) else 0
    ]
    return label

# Función que convierte las imágenes y etiquetas en tensores adecuados para el entrenamiento
def image_data_generator(df, image_folder, batch_size=32):
    images = []
    labels = []

    for index, row in df.iterrows():
        img_id = row['des_filename']
        img_path = os.path.join(image_folder, f"{img_id}")

        try:
            img_array = preprocess_image(img_path)
            images.append(img_array)
        except Exception as e:
            print(f"Error al cargar la imagen {img_path}: {e}")
            continue

        label = load_labels(row)
        labels.append(label)

        # Cuando se alcanza el tamaño del lote, se devuelve el lote
        if len(images) == batch_size:
            labels_one_hot = [to_categorical(np.array(labels)[:, i], num_classes=10000) for i in range(len(labels[0]))]
            yield np.array(images), tuple(labels_one_hot)  # Devolvemos una tupla
            images = []
            labels = []

    # Devolver el último lote si hay imágenes restantes
    if len(images) > 0:
        labels_one_hot = [to_categorical(np.array(labels)[:, i], num_classes=10000) for i in range(len(labels[0]))]
        yield np.array(images), tuple(labels_one_hot)  # Devolvemos una tupla


## Build

In [3]:
# Crear el modelo base de ResNet50 sin la capa superior
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Agregar las capas para la predicción de las características
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Añadir una capa de salida para cada característica
output_silhouette = Dense(10000, activation='softmax', name='silhouette_type')(x)
output_waist = Dense(10000, activation='softmax', name='waist_type')(x)
output_neck_lapel = Dense(10000, activation='softmax', name='neck_lapel_type')(x)
output_sleeve_length = Dense(10000, activation='softmax', name='sleeve_length_type')(x)
output_toecap = Dense(10000, activation='softmax', name='toecap_type')(x)
output_closure = Dense(10000, activation='softmax', name='closure_placement')(x)
output_cane_height = Dense(10000, activation='softmax', name='cane_height_type')(x)
output_heel_shape = Dense(10000, activation='softmax', name='heel_shape_type')(x)
output_knit = Dense(10000, activation='softmax', name='knit_structure')(x)
output_length = Dense(10000, activation='softmax', name='length_type')(x)
output_woven = Dense(10000, activation='softmax', name='woven_structure')(x)

# Crear el modelo con múltiples salidas
model = Model(inputs=base_model.input, outputs=[output_silhouette, output_waist, output_neck_lapel,
                                                output_sleeve_length, output_toecap, output_closure,
                                                output_cane_height, output_heel_shape, output_knit,
                                                output_length, output_woven])

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] * 11)


## Train

In [ ]:
# Uso de tf.data para generar los lotes de imágenes y etiquetas
train_dataset = tf.data.Dataset.from_generator(
    lambda: image_data_generator(df_subset, image_folder),
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        (tf.TensorSpec(shape=(None, 10000), dtype=tf.float32),) * 11  # Definir las salidas como una tupla de tensores
    )
)

# Ajustar el número de pasos por época y el tamaño de los lotes
steps_per_epoch = len(df_subset) // 32

# Entrenamiento del modelo
model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=5)

Epoch 1/5
231/312 ━━━━━━━━━━━━━━━━━━━━ 2:28 2s/step - cane_height_type_accuracy: 0.9418 - cane_height_type_loss: 0.7896 - closure_placement_accuracy: 0.5433 - closure_placement_loss: 1.7100 - heel_shape_type_accuracy: 0.9124 - heel_shape_type_loss: 0.9906 - knit_structure_accuracy: 0.8011 - knit_structure_loss: 1.2198 - length_type_accuracy: 0.4541 - length_type_loss: 2.0908 - loss: 17.4680 - neck_lapel_type_accuracy: 0.4202 - neck_lapel_type_loss: 2.6241 - silhouette_type_accuracy: 0.3645 - silhouette_type_loss: 2.5258 - sleeve_length_type_accuracy: 0.6173 - sleeve_length_type_loss: 1.5744 - toecap_type_accuracy: 0.9249 - toecap_type_loss: 0.8601 - waist_type_accuracy: 0.7408 - waist_type_loss: 1.3447 - woven_structure_accuracy: 0.4594 - woven_structure_loss: 1.7381

## Test

### Imports

In [16]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import os

### Test dataset

In [ ]:
# Cargar el archivo CSV y tomar una muestra aleatoria de 7200 filas
test_data = pd.read_csv('../archive/test_data.csv')
sample_data = test_data.sample(n=7500, random_state=42)  # Tomar una muestra aleatoria

# Prepara una función para cargar y procesar las imágenes
def load_and_process_image(image_path, target_size=(224, 224)):
    try:
        img = image.load_img(image_path, target_size=target_size)  # Cargar la imagen
        img_array = image.img_to_array(img)  # Convertirla a un array
        img_array = np.expand_dims(img_array, axis=0)  # Añadir la dimensión del batch
        img_array = preprocess_input(img_array)  # Preprocesamiento específico de ResNet50
        return img_array
    except (OSError, ValueError) as e:
        print(f"Error al cargar la imagen: {image_path} - {e}")
        return None  # Retorna None si la imagen no es válida

# Función para realizar la predicción
def predict_image(model, image_path):
    img_array = load_and_process_image(image_path)
    if img_array is not None:
        prediction = model.predict(img_array)  # Realizar la predicción
        return prediction
    else:
        return None  # No realiza predicción si la imagen no es válida

# Prepara una lista para almacenar las predicciones
predictions = []

# Ruta de la carpeta donde se encuentran las imágenes (ajusta según tu estructura)
image_folder = '../archive/images/images'

### Predictions

In [ ]:
# Recorremos cada fila del CSV y realizamos la predicción para cada imagen
for _, row in test_data.iterrows():
    image_filename = row['des_filename']
    image_path = os.path.join(image_folder, image_filename)
    
    if os.path.exists(image_path):
        # Realizar la predicción para la imagen
        prediction = predict_image(model, image_path)
        
        if prediction is not None:
            # Si la predicción es válida, procesarla
            if isinstance(prediction, list):
                prediction_list = prediction
            else:
                # Si no es una lista, usar flatten si es un array de NumPy o algo similar
                prediction_list = prediction.flatten().tolist()
            
            # Agregar la predicción junto con los demás datos de la fila
            predictions.append({
                'test_id': row['test_id'],
                'predictions': prediction_list
            })
        else:
            # Si la predicción es inválida, agregar 'INVALID'
            predictions.append({
                'test_id': row['test_id'],
                'predictions': 'INVALID'
            })
    else:
        print(f"Imagen {image_filename} no encontrada en {image_folder}")

# Convertir las predicciones a un DataFrame para fácil visualización
predictions_df = pd.DataFrame(predictions)

# Guardar las predicciones en un archivo CSV (opcional)
predictions_df.to_csv('predictions.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━

AttributeError: 'NoneType' object has no attribute 'flatten'

## Rewrite

In [ ]:
# Cargar los datos, omitiendo la primera fila
predictions = pd.read_csv('predictions.csv', header=None, names=['test_id', 'des_value'], skiprows=1)

# Crear un diccionario de reemplazo de los valores numéricos por sus descripciones (si es necesario, agrega la carga de attributes_data)
attributes_data = pd.read_csv('../archive/attribute_data.csv', header=None, names=['cod_modelo_color', 'attribute_name', 'cod_value', 'des_value'], skiprows=1)

# Función para limpiar 'test_id'
def clean_test_id(test_id):
    parts = test_id.split('__')
    if len(parts) > 1 and '-' in parts[0]:
        prefix, suffix = parts[0].split('-')[0].split('_')[:2]
        attribute = parts[1]
        return f"{prefix}_{suffix}_{attribute}"
    return None

# Aplicar la función para limpiar IDs
predictions['cleaned_test_id'] = predictions['test_id'].apply(clean_test_id)

# Filtrar filas con IDs válidos
predictions = predictions[predictions['cleaned_test_id'].notnull()]

# Eliminar filas duplicadas basadas en 'cleaned_test_id'
unique_predictions = predictions.drop_duplicates(subset='cleaned_test_id', keep='first')

# Guardar el archivo resultante en submission.csv
unique_predictions[['cleaned_test_id', 'des_value']].to_csv('submission.csv', index=False)
